# Derive Top N Portfolio

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

In [2]:
import os
import sys
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from utils.modules.etl.load import sparkDBwls as sdb
# from utils.modules.etl.transform import sparkCleanNRich as scne
from mining.modules.assets.etp import dailyTopN as topN
# from utils.modules.ml.timeseries import rollingstats as stats

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    topN = importlib.reload(topN)
    sdb = importlib.reload(sdb)
#     scne = importlib.reload(scne)
#     stats= importlib.reload(stats)
    
__desc__ = "analyze crypto market capitalization time series data"
clsSDB = sdb.SQLWorkLoads(desc=__desc__)
# clsSCNR=scne.Transformer(desc=__desc__)
clsMPT =topN.WeightedPortfolio(desc=__desc__)
# clsStat=stats.RollingStats(desc=__desc__)
''' optional - if not specified class will use the default values '''
# prop_kwargs = {"WRITE_TO_TMP":True,   # necessary to emulate the etl dag
#               }
print("\nClass initialization and load complete!")

All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All functional SPARKDBWLS-libraries in LOAD-package of ETL-module imported successfully!
All functional DAILYTOPN-libraries in ETP-package of ASSETS-module imported successfully!
All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All functional DAILYTOPN-libraries in ETP-package of ASSETS-module imported successfully!
All functional SPARKDBWLS-libraries in LOAD-package of ETL-module imported successfully!
All functional SPARKCLEANNRICH-libraries in TRANSFORM-package of ETL-module imported successfully!
All functional SPARKNOSQLWLS-libraries in LOAD-package of ETL-module imported successfully!
sparkNoSQLwls Class initialization complete
dailyTopN Class initialization complete

Class initialization and load complete!


## Read data from mcap_past
We apply a query to select assets with mcap > 1.0 million. Any missing values are imputed with the mean value.

In [4]:
_from_date = '2022-01-05'
_to_date = '2022-01-10'
_db_name = "warehouse.mcap_past"
_kwargs={}
# _query = "select * from warehouse.mcap_past "+\
#         f"where mcap_date between '{_from_date}' and '{_to_date}' "+\
#         f"and mcap_value > 1000000 and asset_name in "+\
#         "('brg_x','_crdn','avme','atri','ethix','hoge','xpx','wabi','dmg','mintme','chart')"
_query =f"select * from {_db_name} "+\
        f"where mcap_date between '{_from_date}' and '{_to_date}' "+\
        f"and mcap_value > 1000000 and log_ror is not null"

mcap_sdf = clsMPT.read_ror(select=_query,**_kwargs)

print("Loaded %d rows and %d columns" % (mcap_sdf.count(),len(mcap_sdf.columns)))

Wait a moment, retrieving data ...
23/03/07 08:11:37 WARN Utils: Your hostname, FarmRaiderTester resolves to a loopback address: 127.0.1.1; using 192.168.124.15 instead (on interface enp2s0)
23/03/07 08:11:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/03/07 08:11:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/07 08:11:39 WARN FileSystem: Cannot load filesystem: java.util.ServiceConfigurationError: org.apache.hadoop.fs.FileSystem: com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem Unable to get public no-arg constructor
23/03/07 08:11:39 WARN FileSystem: java.lang.NoClassDefFoundError: com/google/api/client/auth/oauth2/Credential
23/03/07 08:11:39 WARN FileSystem: java.lang.ClassNotFoundException: com.google.api.client.auth.oauth2.Credential


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Loaded 10908 rows and 17 columns


In [5]:
from pyspark.sql import functions as F

mcap_sdf.select(
    F.col('mcap_past_pk'),F.col('mcap_date'),
    F.col('asset_name'),F.col('mcap_value'),F.col('log_ror'))\
    .sort(F.col('mcap_date'))\
    .show(n=10)

+------------+-------------------+----------+--------------------+------------------+
|mcap_past_pk|          mcap_date|asset_name|          mcap_value|           log_ror|
+------------+-------------------+----------+--------------------+------------------+
|       16328|2022-01-05 00:00:00|     audio|851781048.4787650...|0.6928591390799810|
|       63439|2022-01-05 00:00:00|      dose|8672474.266345850...|0.6763315876313835|
|       97873|2022-01-05 00:00:00|       kus|3176017.219590980...|0.6998238419681013|
|       39730|2022-01-05 00:00:00|      drct|2244212.912972310...|0.6575184485114410|
|       28910|2022-01-05 00:00:00|       l3p|1317788.022696750...|0.7187257420450860|
|      151661|2022-01-05 00:00:00|        la|9276804.041408930...|0.6895376741222805|
|        3694|2022-01-05 00:00:00|  babydoge|486963873.8483450...|0.6986166954553891|
|       14630|2022-01-05 00:00:00|      dino|6450788.559264860...|0.6771463472778994|
|       55686|2022-01-05 00:00:00|      kuma|13786922.

## Weighted Portfolio

In [6]:
_cols={
    "PRIMARYKEY":'mcap_past_pk',
    "NAMECOLUMN":'asset_name',
    "DATECOLUMN":'mcap_date',
    "NUMCOLUMN" :'log_ror',
    "MCAPCOLUMN":'mcap_value',
    "WEIGHTCOLUMN":'weights',
    "MCAPSOURCE":'source',
}
_l_exp_wts,_cols_dict=clsMPT.get_weighted_mpt(
    data=mcap_sdf,
    cols_dict=_cols,
    topN=3,
    size=5,
    **_kwargs,
)
_l_exp_wts[:2]

[{'source': 'warehouse.mcap_past',
  'mcap_past_pk': [91391, 124438, 78462],
  'mcap_date': datetime.datetime(2022, 1, 9, 0, 0),
  'asset_name': ['btcs', 'nabox', 'yaxis'],
  'log_ror': [Decimal('2.9321703953454180'),
   Decimal('1.9620722641637465'),
   Decimal('1.4506834179632306')],
  'weights': [0.6789370563680265, 0.06345221733977897, 0.25761072629219456],
  'mcap_value': [Decimal('23756320.2128301000000000'),
   Decimal('66498522.3461270000000000'),
   Decimal('3036867.5895900100000000')]},
 {'source': 'warehouse.mcap_past',
  'mcap_past_pk': [130969, 111435, 62127],
  'mcap_date': datetime.datetime(2022, 1, 8, 0, 0),
  'asset_name': ['l2', 'falcx', 'dana'],
  'log_ror': [Decimal('1.5337095512357850'),
   Decimal('1.3188759637834600'),
   Decimal('1.2139873905404268')],
  'weights': [0.6042672952906084, 0.14590263443562979, 0.24983007027376175],
  'mcap_value': [Decimal('10755738.7793784000000000'),
   Decimal('2651804.8528023700000000'),
   Decimal('26502663.9578732000000000')]}

## Write MPT to MongoDB

* Collection name = "mpt."+date(YYYY-MM-DD)
* document structure: \_id, date, asset, mcap.value, mcap.weight, mcap.ror

In [7]:
_kwargs = {
    "DESTINDBNAME":'tip-daily-mpt',
    "COLLPREFIX" : 'mpt'
}
mpt_list_ = clsMPT.write_mpt_to_db(
    mpt_data=_l_exp_wts,
    cols_dict=_cols,
    **_kwargs,
)
print("Upsert %d documents" % len(mpt_list_))

Upsert 6 documents
